In [18]:
# Import login values from config
from configparser import *

In [19]:
# Import packages needed
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.snowpark import Window
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import numpy as np

In [20]:
#we save credentials in a credentials_dict dictionary
import sys, json
with open("connection_parameters_prod.json") as jsonfile:
    credentials_dict = json.load(jsonfile)

In [21]:
#we connect to snowflake using credentials_dict and declare a session
def snowpark_session_create():
    session = Session.builder.configs(credentials_dict).create()
    return session

In [22]:
#we create a session
session = snowpark_session_create()

In [23]:
#we declare what db and what schema we will be using, we can change this within sql
session.use_database("SNF_MDRAZ")
session.use_schema("bronze")

In [66]:
#we are defining an sql query which will be used to create a bronze table where data will come from staging
#IF NOT EXISTS
session.sql("""
    create or replace TABLE SNF_MDRAZ.BRONZE.KNA1 (
	INDEX VARCHAR(16777216),
	KUNNR VARCHAR(16777216),
	GENDE VARCHAR(16777216),
	NAME1 VARCHAR(16777216),
	NAME2 VARCHAR(16777216),
	STRAS VARCHAR(16777216),
	ORT01 VARCHAR(16777216),
	PSTLZ VARCHAR(16777216),
	LAND1 VARCHAR(16777216),
	LAND2 VARCHAR(16777216),
	RSCON VARCHAR(16777216),
	TELF1 VARCHAR(16777216),
	REGIO VARCHAR(16777216),
	BIRTH DATE,
	LATI VARCHAR(16777216),
	LONG VARCHAR(16777216),
	LASTMODIFIED DATE
);
"""
).collect()                        

[Row(status='Table KNA1 successfully created.')]

In [13]:
#we execute sql query for bronze creation and use collect() to force execution
#session.sql(create_or_replace_table).collect()

[Row(status='KNA1 already exists, statement succeeded.')]

In [133]:
#we are selecting data from staging table kna1
source_table = """select
    index,
    kunnr,
    gende,
    name1,
    name2,
    stras,
    ort01,
    pstlz,
    land1,
    land2,
    rscon,
    telf1,
    regio,
    to_date(birth,'YYYY/MM/DD') birth,
    lati,
    long,
    current_date() as LASTMODIFIED from staging.kna1;"""
df_source = session.sql(source_table).to_pandas()

In [134]:
#we are listing what values we want to use to replace NaNs and Nulls
values = {
    'INDEX': -99,
    'KUNNR': 'NA',
    'GENDE': 'NA', 
    'NAME1': 'NA', 
    'NAME2': 'NA', 
    'STRAS': 'NA', 
    'ORT01': 'NA', 
    'PSTLZ': 'NA', 
    'LAND1': 'NA', 
    'LAND2': 'NA', 
    'RSCON': 'NA', 
    'TELF1': 'NA', 
    'REGIO': -99, 
    'BIRTH': '1900/01/01', 
    'LATI': 0, 
    'LONG': 0, 
    'LASTMODIFIED' : '1900/01/01'
}
df_source.fillna(value=values,inplace=True)

In [135]:
#since couple of columns have wrong types, here we are changing these types to what we need
#also couple of data has ' in them, we are replacing them with '' to avoid accidental break in string
df_source['INDEX'] = df_source['INDEX'].astype('int64')
df_source['REGIO'] = df_source['REGIO'].astype('str')
df_source['GENDE'] = df_source['GENDE'].astype('str')
df_source['BIRTH'] = pd.to_datetime(df_source['BIRTH'],format='ISO8601').dt.strftime('%Y-%m-%d')
df_source['ORT01'] = df_source['ORT01'].replace("'","''")
df_source = df_source.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [136]:
#another way of inserting data into snowflake
try:
    session.sql("BEGIN TRANSACTION")
    spDF = session.write_pandas(
        df = df_source,
        table_name="kna1",
        database="snf_mdraz",
        schema="bronze",
        quote_identifiers=False,
        overwrite=True
    )
    session.sql("COMMIT")
except Exception as e:
        session.sql("Rollback")
        print(f"Error encountered: {e}. All inserts have been rolled back.")  

In [36]:
#we are creating a temp table with
temp_tbl = "temp_kna1_cleaned"
df_temp = session.create_dataframe(df_source)
df_temp.write.mode("overwrite").saveAsTable(temp_tbl)

In [37]:
session.sql("truncate bronze.kna1;").collect()
bulk_insert_sql = f"""
                    insert into snf_mdraz.bronze.kna1 (INDEX,KUNNR,GENDE,NAME1,NAME2,STRAS,ORT01,PSTLZ,LAND1,LAND2,RSCON,TELF1,REGIO,BIRTH,LATI,LONG,LASTMODIFIED) 
                    select 
                        index,
                        kunnr,    
                        gende,    
                        name1,    
                        name2,    
                        stras,    
                        ort01,    
                        pstlz,    
                        land1,    
                        land2,    
                        rscon,    
                        telf1,    
                        regio,    
                        birth,    
                        lati,    
                        long,
                        LASTMODIFIED
                    from {temp_tbl}
                    """
try:
    session.sql("BEGIN TRANSACTION")
    session.sql(bulk_insert_sql).collect()
    session.sql("COMMIT")
except Exception as e:
        session.sql("Rollback")
        print(f"Error encountered: {e}. All inserts have been rolled back.")  
session.sql(f"DROP TABLE IF EXISTS {temp_tbl}").collect()

[Row(status='TEMP_KNA1_CLEANED successfully dropped.')]

In [97]:
session.sql("""
    create or replace TABLE SNF_MDRAZ.SILVER.CUSTOMER (
	ID NUMBER(38,0),
	CUSTOMERID VARCHAR(16777216),
	GENDER VARCHAR(16777216),
	FIRSTNAME VARCHAR(16777216),
	LASTNAME VARCHAR(16777216),
	STREETADDRESS VARCHAR(16777216),
	CITY VARCHAR(16777216),
	POSTALCODE VARCHAR(16777216),
	COUNTRYCODE VARCHAR(16777216),
	COUNTRYNAME VARCHAR(16777216),
	EMAILADDRESS VARCHAR(16777216),
	PHONENUMBER VARCHAR(16777216),
	PHONECOUNTRYCODE NUMBER(38,0),
	DATEOFBIRTH DATE,
	LATITUDE NUMBER(10,6),
	LONGITUDE NUMBER(10,6),
	LASTMODIFIED DATE
);
"""
).collect()

[Row(status='Table CUSTOMER successfully created.')]

In [138]:
#we are selecting data from staging table kna1
source_table = """select
    index,
    kunnr,
    gende,
    name1,
    name2,
    stras,
    ort01,
    pstlz,
    land1,
    land2,
    rscon,
    telf1,
    regio,
    birth,
    lati,
    long,
    LASTMODIFIED from bronze.kna1;"""
df_bronze = session.sql(source_table).to_pandas()

In [ ]:
df_bronze= df_bronze.sort_values(by=['LASTMODIFIED'], ascending=False)
df_bronze['rn'] = df_bronze.groupby(['INDEX','KUNNR']).cumcount()+1
df_bronze[df_bronze.rn == 1]
df_bronze.reset_index(inplace=True)

In [143]:
target = session.table("SNF_MDRAZ.SILVER.CUSTOMER")
df_sp_bronze = session.create_dataframe(df_bronze)

merged = target.merge(
    source=df_sp_bronze,
    join_expr=(target['ID'] == df_sp_bronze['INDEX']),
    clauses= [
        when_matched().update(
            {
            "CUSTOMERID": df_sp_bronze['KUNNR'],
            "GENDER": df_sp_bronze['GENDE'], 
            "FIRSTNAME": df_sp_bronze['NAME1'], 
            "LASTNAME": df_sp_bronze['NAME2'], 
            "STREETADDRESS": df_sp_bronze['STRAS'], 
            "CITY": df_sp_bronze['ORT01'], 
            "POSTALCODE": df_sp_bronze['PSTLZ'], 
            "COUNTRYCODE": df_sp_bronze['LAND1'], 
            "COUNTRYNAME": df_sp_bronze['LAND2'], 
            "EMAILADDRESS": df_sp_bronze['RSCON'], 
            "PHONENUMBER": df_sp_bronze['TELF1'], 
            "PHONECOUNTRYCODE": df_sp_bronze['REGIO'], 
            "DATEOFBIRTH": df_sp_bronze['BIRTH'], 
            "LATITUDE": df_sp_bronze['LATI'], 
            "LONGITUDE": df_sp_bronze['LONG'], 
            "LASTMODIFIED": df_sp_bronze['LASTMODIFIED']
            }
        ),
        when_not_matched().insert(
            {
            "ID": df_sp_bronze['INDEX'], 
            "CUSTOMERID": df_sp_bronze['KUNNR'],
            "GENDER": df_sp_bronze['GENDE'], 
            "FIRSTNAME": df_sp_bronze['NAME1'], 
            "LASTNAME": df_sp_bronze['NAME2'], 
            "STREETADDRESS": df_sp_bronze['STRAS'], 
            "CITY": df_sp_bronze['ORT01'], 
            "POSTALCODE": df_sp_bronze['PSTLZ'], 
            "COUNTRYCODE": df_sp_bronze['LAND1'], 
            "COUNTRYNAME": df_sp_bronze['LAND2'], 
            "EMAILADDRESS": df_sp_bronze['RSCON'], 
            "PHONENUMBER": df_sp_bronze['TELF1'], 
            "PHONECOUNTRYCODE": df_sp_bronze['REGIO'], 
            "DATEOFBIRTH": df_sp_bronze['BIRTH'], 
            "LATITUDE": df_sp_bronze['LATI'], 
            "LONGITUDE": df_sp_bronze['LONG'], 
            "LASTMODIFIED": df_sp_bronze['LASTMODIFIED']
            }
        )
    ]
)

In [144]:
merged

MergeResult(rows_inserted=429187, rows_updated=0, rows_deleted=0)